# Clean dim_products từ Silver lên Gold

## Mục tiêu
- Load dữ liệu từ Silver.dim_products → Gold.dim_products
- Clean và chuẩn hóa dữ liệu cho EDA
- Đảm bảo data quality cho Gold layer

## Quy trình 6 bước
1. **Load from Silver** - Đọc dữ liệu từ Silver database
2. **Data Quality Check** - Phân tích chất lượng dữ liệu
3. **Data Cleaning** - Xử lý các vấn đề về dữ liệu
4. **Data Validation** - Kiểm tra kết quả sau khi clean
5. **Load to Gold** - Load vào Gold database
6. **Create Dictionary** - Tạo data dictionary


## Bước 1: Import và Setup


In [5]:
# Import các thư viện cần thiết
import os
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from dotenv import load_dotenv
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Load biến môi trường từ file .env
load_dotenv()

# Lấy thông tin kết nối database
DB_USER = os.getenv("DB_USER")
DB_PASS = os.getenv("DB_PASS")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_SILVER = os.getenv("DB_SILVER")
DB_GOLD = os.getenv("DB_GOLD")

# Tạo kết nối tới Silver và Gold database
silver_engine = create_engine(f"mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_SILVER}")
gold_engine = create_engine(f"mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_GOLD}")

print("Đã kết nối Silver và Gold database thành công")
print(f"Silver DB: {DB_SILVER}")
print(f"Gold DB: {DB_GOLD}")
print(f"Thời gian bắt đầu: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


Đã kết nối Silver và Gold database thành công
Silver DB: winner_silver
Gold DB: winner_gold
Thời gian bắt đầu: 2025-10-17 23:58:46


## Bước 2: Load dữ liệu từ Silver


In [6]:
# Load dữ liệu dim_products từ Silver database
print("=== LOADING DIM_PRODUCTS FROM SILVER ===")

# Load toàn bộ bảng dim_products
products_df = pd.read_sql("SELECT * FROM dim_products", silver_engine)

# Hiển thị thông tin cơ bản về dataset
print(f"Shape: {products_df.shape}")
print(f"Columns: {list(products_df.columns)}")
print(f"Memory usage: {products_df.memory_usage(deep=True).sum() / 1024 / 1024:.2f} MB")

# Hiển thị sample data
print("\n=== SAMPLE DATA ===")
print(products_df.head(3))

# Hiển thị data types
print("\n=== DATA TYPES ===")
print(products_df.dtypes)


=== LOADING DIM_PRODUCTS FROM SILVER ===
Shape: (37, 19)
Columns: ['product_id', 'name', 'custom_id', 'shop_id', 'type', 'category_ids', 'category_name', 'creator_id', 'removed', 'lifecycle_days', 'min_price', 'max_price', 'total_variations', 'total_quantity', 'remain_quantity', 'colors', 'sizes', 'inserted_at', 'updated_at']
Memory usage: 0.03 MB

=== SAMPLE DATA ===
                             product_id              name  \
0  3ec3103b-ee8e-484a-abf3-d542f4487c18  Áo Thu Đông.W033   
1  6baccc22-e6a9-4fde-9bd9-52c7d2e1c4d9     Áo Khoác.W032   
2  3508fe85-4a72-4965-aec6-7a908553617a  Áo Thu Đông.W029   

                   custom_id    shop_id     type category_ids category_name  \
0  WINNER - Áo thu đông.W033  230361475  product   1290005112       Tam Anh   
1     WINNER - Áo Khoác.W032  230361475  product   1290028415      Lê Phong   
2  WINNER - Áo thu đông.W029  230361475  product   1290021501     Truong.NQ   

                             creator_id  removed  lifecycle_days  m

## Bước 3: Data Quality Check


In [7]:
# Phân tích chất lượng dữ liệu
print("=== DATA QUALITY ANALYSIS ===")

# 1. Null ratio analysis
print("\n1. NULL RATIO ANALYSIS")
null_ratio = products_df.isnull().mean().sort_values(ascending=False)
print("Tỷ lệ null theo cột:")
for col, ratio in null_ratio.items():
    print(f"  {col}: {ratio:.3f} ({ratio*100:.1f}%)")

# 2. Duplicate check
print(f"\n2. DUPLICATE CHECK")
if 'product_id' in products_df.columns:
    dup_count = products_df["product_id"].duplicated().sum()
    print(f"Số lượng product_id trùng lặp: {dup_count}")

# 3. Value analysis cho các trường quan trọng
print(f"\n3. VALUE ANALYSIS")

# String fields analysis
string_fields = ['product_name', 'product_category', 'product_brand']
for field in string_fields:
    if field in products_df.columns:
        print(f"\n--- {field.upper()} ANALYSIS ---")
        value_counts = products_df[field].value_counts(dropna=False).head(10)
        print(value_counts)
        print(f"Null count: {products_df[field].isnull().sum()}")

# 4. Overall quality summary
print(f"\n4. OVERALL QUALITY SUMMARY")
total_columns = len(products_df.columns)
high_null_columns = len(null_ratio[null_ratio > 0.7])
print(f"Total columns: {total_columns}")
print(f"Columns with >70% null: {high_null_columns}")
print(f"Data quality score: {((total_columns - high_null_columns) / total_columns * 100):.1f}%")


=== DATA QUALITY ANALYSIS ===

1. NULL RATIO ANALYSIS
Tỷ lệ null theo cột:
  category_name: 0.135 (13.5%)
  category_ids: 0.135 (13.5%)
  total_quantity: 0.081 (8.1%)
  remain_quantity: 0.081 (8.1%)
  custom_id: 0.000 (0.0%)
  product_id: 0.000 (0.0%)
  name: 0.000 (0.0%)
  creator_id: 0.000 (0.0%)
  removed: 0.000 (0.0%)
  type: 0.000 (0.0%)
  shop_id: 0.000 (0.0%)
  min_price: 0.000 (0.0%)
  lifecycle_days: 0.000 (0.0%)
  total_variations: 0.000 (0.0%)
  max_price: 0.000 (0.0%)
  colors: 0.000 (0.0%)
  sizes: 0.000 (0.0%)
  inserted_at: 0.000 (0.0%)
  updated_at: 0.000 (0.0%)

2. DUPLICATE CHECK
Số lượng product_id trùng lặp: 0

3. VALUE ANALYSIS

4. OVERALL QUALITY SUMMARY
Total columns: 19
Columns with >70% null: 0
Data quality score: 100.0%


## Bước 4: Data Cleaning


In [8]:
# Xử lý dữ liệu products
print("=== CLEANING PRODUCTS DATA ===")

# Tạo bản copy để xử lý
products_clean = products_df.copy()

# 1. Xóa cột category_ids
print("1. Xóa cột category_ids...")
if 'category_ids' in products_clean.columns:
    products_clean = products_clean.drop(columns=['category_ids'])
    print("Đã xóa cột category_ids thành công")
else:
    print("Cột category_ids không tồn tại trong dataset")

# 2. Fillna cột category_name = "Le Van Duy"
print("\n2. Fillna cột category_name...")
if 'category_name' in products_clean.columns:
    # Hiển thị trạng thái trước khi xử lý
    null_count_before = products_clean['category_name'].isnull().sum()
    print(f"Null count trước khi xử lý: {null_count_before}")
    
    # Fillna với "Le Van Duy"
    products_clean['category_name'] = products_clean['category_name'].fillna("Le Van Duy")
    
    # Hiển thị kết quả
    null_count_after = products_clean['category_name'].isnull().sum()
    print(f"Null count sau khi xử lý: {null_count_after}")
    print("Đã fillna cột category_name với 'Le Van Duy'")
else:
    print("Cột category_name không tồn tại trong dataset")

# Hiển thị kết quả
print(f"\n=== KẾT QUẢ SAU KHI CLEANING ===")
print(f"Shape: {products_clean.shape}")
print(f"Columns: {list(products_clean.columns)}")
print(f"Null values: {products_clean.isnull().sum().sum()}")

print(f"\nSample data sau khi cleaning:")
print(products_clean.head(5))


=== CLEANING PRODUCTS DATA ===
1. Xóa cột category_ids...
Đã xóa cột category_ids thành công

2. Fillna cột category_name...
Null count trước khi xử lý: 5
Null count sau khi xử lý: 0
Đã fillna cột category_name với 'Le Van Duy'

=== KẾT QUẢ SAU KHI CLEANING ===
Shape: (37, 18)
Columns: ['product_id', 'name', 'custom_id', 'shop_id', 'type', 'category_name', 'creator_id', 'removed', 'lifecycle_days', 'min_price', 'max_price', 'total_variations', 'total_quantity', 'remain_quantity', 'colors', 'sizes', 'inserted_at', 'updated_at']
Null values: 6

Sample data sau khi cleaning:
                             product_id              name  \
0  3ec3103b-ee8e-484a-abf3-d542f4487c18  Áo Thu Đông.W033   
1  6baccc22-e6a9-4fde-9bd9-52c7d2e1c4d9     Áo Khoác.W032   
2  3508fe85-4a72-4965-aec6-7a908553617a  Áo Thu Đông.W029   
3  358156d8-2777-4aab-b584-a26813fb4b95  Áo Thu Đông.W031   
4  6afa1a3e-4e23-46e8-a434-3018fdaebf12  Áo Thu Đông.W030   

                   custom_id    shop_id     type categ

## Bước 5: Load vào Gold Database


In [9]:
# Load vào Gold Database
print("=== LOADING TO GOLD DATABASE ===")

from sqlalchemy.types import BigInteger, Integer, String, Text, DateTime, Float

# Định nghĩa schema mapping cho MySQL
dtype_mapping = {}

# ID fields - String (để xử lý mixed data types)
id_fields = ["product_id"]
for field in id_fields:
    if field in products_clean.columns:
        dtype_mapping[field] = String(100)

# String fields - VARCHAR(255)
string_fields = ["product_name", "category_name", "product_brand"]
for field in string_fields:
    if field in products_clean.columns:
        dtype_mapping[field] = String(255)

# Text fields - Text()
text_fields = ["product_description"]
for field in text_fields:
    if field in products_clean.columns:
        dtype_mapping[field] = Text()

# Numeric fields
float_fields = ["product_price", "product_quantity"]
for field in float_fields:
    if field in products_clean.columns:
        dtype_mapping[field] = Float()

# DateTime fields
datetime_fields = ["created_at", "updated_at"]
for field in datetime_fields:
    if field in products_clean.columns:
        dtype_mapping[field] = DateTime()

print(f"Schema mapping defined for {len(dtype_mapping)} columns")

# Load vào Gold database
table_name = "dim_products"

try:
    products_clean.to_sql(
        table_name,
        con=gold_engine,
        if_exists="replace",  # Ghi đè dữ liệu cũ
        index=False,
        dtype=dtype_mapping
    )
    
    print(f"Đã load {products_clean.shape[0]} records vào Gold: {table_name}")
    
    # Verify load
    verification_query = f"SELECT COUNT(*) as count FROM {table_name}"
    verification_result = pd.read_sql(verification_query, gold_engine)
    loaded_count = verification_result['count'].iloc[0]
    
    print(f"Verification: {loaded_count} records trong database")
    print(f"Schema: {len(dtype_mapping)} columns với explicit typing")
    
    # Sample data từ database
    sample_query = f"SELECT * FROM {table_name} LIMIT 5"
    sample_db = pd.read_sql(sample_query, gold_engine)
    print(f"\nSample data từ Gold database:\n{sample_db}")
    
except Exception as e:
    print(f"Error loading to Gold: {str(e)}")
    raise

print(f"\nLoad dim_products completed successfully!")
print(f"Ready for EDA and analytics")


=== LOADING TO GOLD DATABASE ===
Schema mapping defined for 3 columns
Đã load 37 records vào Gold: dim_products
Verification: 37 records trong database
Schema: 3 columns với explicit typing

Sample data từ Gold database:
                             product_id              name  \
0  3ec3103b-ee8e-484a-abf3-d542f4487c18  Áo Thu Đông.W033   
1  6baccc22-e6a9-4fde-9bd9-52c7d2e1c4d9     Áo Khoác.W032   
2  3508fe85-4a72-4965-aec6-7a908553617a  Áo Thu Đông.W029   
3  358156d8-2777-4aab-b584-a26813fb4b95  Áo Thu Đông.W031   
4  6afa1a3e-4e23-46e8-a434-3018fdaebf12  Áo Thu Đông.W030   

                   custom_id    shop_id     type category_name  \
0  WINNER - Áo thu đông.W033  230361475  product       Tam Anh   
1     WINNER - Áo Khoác.W032  230361475  product      Lê Phong   
2  WINNER - Áo thu đông.W029  230361475  product     Truong.NQ   
3  WINNER - Áo thu đông.W031  230361475  product       Tam Anh   
4  WINNER - Áo thu đông.W030  230361475  product       Tam Anh   

               

## Bước 6: Tạo Data Dictionary cho Gold


In [10]:
# Tạo Data Dictionary cho Gold
print("=== CREATING GOLD DATA DICTIONARY ===")

def get_business_meaning_vietnamese(column_name):
    """Get business meaning for each column in Vietnamese"""
    business_meanings = {
        # Định danh & Cơ bản
        "product_id": "Mã định danh sản phẩm duy nhất (khóa chính)",
        
        # Thông tin sản phẩm
        "product_name": "Tên sản phẩm",
        "category_name": "Tên danh mục sản phẩm (đã fillna với 'Le Van Duy')",
        "product_brand": "Thương hiệu sản phẩm",
        "product_description": "Mô tả chi tiết sản phẩm",
        
        # Giá và số lượng
        "product_price": "Giá sản phẩm (VND)",
        "product_quantity": "Số lượng sản phẩm trong kho",
        
        # Thời gian
        "created_at": "Thời gian tạo bản ghi sản phẩm",
        "updated_at": "Thời gian cập nhật bản ghi cuối cùng"
    }
    return business_meanings.get(column_name, "Chưa định nghĩa ý nghĩa business")

# Tạo Data Dictionary
dict_data = []
for col in products_clean.columns:
    col_info = {
        "table_name": "dim_products",
        "column_name": col,
        "dtype": str(products_clean[col].dtype),
        "sql_type": str(dtype_mapping.get(col, "Not defined")),
        "null_count": products_clean[col].isnull().sum(),
        "null_pct": round(products_clean[col].isnull().mean() * 100, 2),
        "unique_count": products_clean[col].nunique(),
        "sample_values": str(products_clean[col].dropna().unique()[:3].tolist()),
        "business_meaning": get_business_meaning_vietnamese(col),
        "extraction_date": datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    }
    dict_data.append(col_info)

data_dictionary = pd.DataFrame(dict_data)

# Hiển thị Data Dictionary
print(f"Generated Data Dictionary for {len(data_dictionary)} columns")
print("\n=== DATA DICTIONARY ===")
print(data_dictionary)

# Lưu vào file Excel
excel_path = "Technical_Document/Gold_dictionary.xlsx"

try:
    from openpyxl import load_workbook
    
    # Kiểm tra file Excel có tồn tại không
    try:
        # Load workbook hiện tại
        wb = load_workbook(excel_path)
        
        # Lấy sheet đầu tiên
        ws = wb.active
        
        # Kiểm tra xem có dữ liệu cũ không
        if ws.max_row > 1:
            print(f"Found existing data in {excel_path}, appending new data...")
        else:
            print(f"File {excel_path} exists but is empty, adding header and data...")
            
    except FileNotFoundError:
        print(f"File {excel_path} not found, creating new file...")
        wb = None
    except Exception as e:
        print(f"Error loading {excel_path}: {str(e)}, creating new file...")
        wb = None
    
    if wb is None:
        # Tạo file mới với header
        data_dictionary.to_excel(excel_path, index=False, sheet_name='Gold_Dictionary')
        print(f"✅ Created new file: {excel_path}")
    else:
        # Append vào file hiện tại
        from openpyxl.utils.dataframe import dataframe_to_rows
        
        # Tìm dòng cuối cùng có dữ liệu
        last_row = ws.max_row
        
        # Thêm dữ liệu mới từ dòng tiếp theo
        for r in dataframe_to_rows(data_dictionary, index=False, header=False):
            last_row += 1
            for c_idx, value in enumerate(r, 1):
                ws.cell(row=last_row, column=c_idx, value=value)
        
        # Lưu file
        wb.save(excel_path)
        print(f"✅ Appended {len(data_dictionary)} rows to: {excel_path}")
    
except Exception as e:
    print(f"❌ Error appending to Data Dictionary: {str(e)}")
    # Fallback: tạo file mới
    try:
        data_dictionary.to_excel(excel_path, index=False)
        print(f"✅ Created new file as fallback: {excel_path}")
    except Exception as e2:
        print(f"❌ Error creating fallback file: {str(e2)}")

# Summary Report
print(f"\n=== GOLD DICTIONARY SUMMARY ===")
print(f"Table: dim_products")
print(f"Total columns: {len(data_dictionary)}")
print(f"Data Dictionary: {excel_path}")
print(f"Created at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

print(f"\n=== SAMPLE DICTIONARY ENTRIES ===")
sample_dict = data_dictionary[['column_name', 'dtype', 'business_meaning']].head(5)
print(sample_dict)


=== CREATING GOLD DATA DICTIONARY ===
Generated Data Dictionary for 18 columns

=== DATA DICTIONARY ===
      table_name       column_name           dtype      sql_type  null_count  \
0   dim_products        product_id          object  VARCHAR(100)           0   
1   dim_products              name          object   Not defined           0   
2   dim_products         custom_id          object   Not defined           0   
3   dim_products           shop_id           int64   Not defined           0   
4   dim_products              type          object   Not defined           0   
5   dim_products     category_name          object  VARCHAR(255)           0   
6   dim_products        creator_id          object   Not defined           0   
7   dim_products           removed           int64   Not defined           0   
8   dim_products    lifecycle_days           int64   Not defined           0   
9   dim_products         min_price         float64   Not defined           0   
10  dim_products